Run this notebook in the same virtual environment with superlinked server
to ensure the same version of `superlinked` framework.

In [5]:
!pip freeze | grep superlinked

superlinked==29.1.0
superlinked-server==1.37.0


In [6]:
from pathlib import Path
import sys

# Determine current working directory
cwd = Path.cwd()

# Adapt logic for your specific project structure
if cwd.name == "intelligent-file-search":
    project_dir = cwd
elif cwd.name == "notebooks":
    project_dir = cwd.parent
else:
    # fallback (e.g., if inside a subfolder deeper in notebooks or dataset)
    project_dir = cwd
    while project_dir.name != "intelligent-file-search" and project_dir != project_dir.parent:
        project_dir = project_dir.parent

superlinked_app_dir = project_dir / "superlinked_app"
assert superlinked_app_dir.exists(), (
    f"{superlinked_app_dir} does not exist\n"
    "Are you sure you are in or below the intelligent-file-search directory?"
)

if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))
    print("project_dir is added to sys.path")
else:
    print("project_dir is already in sys.path")


project_dir is added to sys.path


### This is to use the collection_name=filesearch

In [7]:
import os
os.environ["APP_ID"] = "filesearch"


In [8]:
from superlinked import framework as sl

from superlinked_app.index import index, file_schema
from superlinked_app.query import query
from superlinked_app.api import vector_database

import pandas as pd

/home/biso/development/my_projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/biso/development/my_projects/varie/intelligent-file-search/dataset/categories.json
16:48:01 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/paraphrase-MiniLM-L3-v2


Batches: 100%|██████████| 1/1 [00:00<00:00, 14.07it/s]

16:48:04 superlinked.framework.common.space.embedding.model_based.embedding_engine_manager INFO   Consider caching model dimension.
16:48:04 superlinked.framework.common.space.embedding.model_based.embedding_engine_manager INFO   Consider caching model dimension.
16:48:04 superlinked.framework.dsl.index.index INFO   initialized index


In [9]:
source = sl.InteractiveSource(file_schema)
executor = sl.InteractiveExecutor(
    sources=[source],
    indices=[index],
    vector_database=vector_database,
)
app = executor.run()

16:48:14 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/filesearch/exists "HTTP/1.1 200 OK"
16:48:14 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/filesearch "HTTP/1.1 200 OK"
16:48:15 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/filesearch/index?wait=true "HTTP/1.1 200 OK"
16:48:15 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/filesearch/index?wait=true "HTTP/1.1 200 OK"
16:48:15 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/filesearch/index?wait=true "HTTP/1.1 200 OK"
16:48:15 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collecti

In [10]:
from datetime import datetime
import pandas as pd
def date_to_unix(date_str):
    if date_str is None:
        return None
    return int(datetime.strptime(date_str, "%d/%m/%Y").timestamp())
# Natural language query parameters
params = {
    "natural_query": "Recent PDF files about fashion or sport",
    "limit": 5,
}

# Run the query with natural language interface
result = app.query(query, **params)

# Print the actual interpreted search parameters extracted by NLQ
print("Search Parameters Parsed by NLQ:")
print(result.metadata.search_params)

# Convert the results to a pandas DataFrame
df = sl.PandasConverter.to_pandas(result)

# Convert Unix timestamps to dd/mm/yyyy format in the DataFrame
import pandas as pd
df["Creation_Date"] = pd.to_datetime(df["Creation_Date"], unit="s").dt.strftime("%d/%m/%Y")
df["Last_Modified_Date"] = pd.to_datetime(df["Last_Modified_Date"], unit="s").dt.strftime("%d/%m/%Y")

df

16:48:52 openai._base_client INFO   Retrying request to /chat/completions in 0.446543 seconds
16:48:56 httpx INFO   HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Batches: 100%|██████████| 1/1 [00:00<00:00, 82.13it/s]

16:48:58 superlinked.framework.query.query_dag_evaluator INFO   evaluated query


16:48:59 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/filesearch/points/query "HTTP/1.1 200 OK"
16:48:59 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
Search Parameters Parsed by NLQ:
{'content_query': 'fashion or sport', 'similar_content_weight': 1.0, 'filename_query': 'pdf', 'similar_filename_weight': 1.0, 'limit': 5, 'select_param__': ['Content', 'File_Type', 'Tags', 'Filename', 'Path', 'Size_kB', 'Creation_Date', 'Last_Modified_Date'], 'min_size_kb': None, 'max_size_kb': None, 'min_creation_date': None, 'max_creation_date': None, 'min_modified_date': None, 'max_modified_date': 1609459200, 'filetype_include_all': None, 'filetype_include_any': ['pdf'], 'filetype_exclude': None, 'tags_include_all': None, 'tags_include_any': ['fashion', 'sport'], 'tags_exclude': None, 'natural_query': 'Recent PDF files about fashion or sport', 'system_prompt_param__': "Extract search paramet

,Content,File_Type,Tags,Filename,Path,Size_kB,Creation_Date,Last_Modified_Date,id,similarity_score
0,Model designer trend dress.,[pdf],[fashion],trend.pdf,/focus/impact/sound/trend.pdf,8776.0,20/12/2001,24/01/2014,id_96,0.534662
1,Basketball athlete tennis football championshi...,[pdf],"[sport, fashion, cinema]",basketba.pdf,/argue/basketba.pdf,6608.0,09/11/2012,17/10/2014,id_58,0.525691
2,Award movie scene director scene movie. Footba...,[pdf],"[cinema, sport, cars]",award.pdf,/financial/under/three/award.pdf,8383.0,24/11/2001,16/12/2011,id_20,0.475730
3,Athlete football athlete football athlete bask...,[pdf],[sport],athlete.pdf,/still/everything/economic/athlete.pdf,1114.0,28/05/2010,16/02/2014,id_24,0.428724


In [11]:

# These are the names of the semantic/numeric spaces in the same order as in your index.spaces list
space_names = [
    "content_similarity",
    "filename_similarity",
    "size_score",
    "creation_date_score",
    "modified_date_score"
]

rows = []

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)
    
df = pd.DataFrame(rows)
df


,id,content_similarity,filename_similarity,size_score,creation_date_score,modified_date_score
0,id_96,0.147602,0.184916,0.0,0.0,0.202144
1,id_58,0.160317,0.157782,0.0,0.0,0.207591
2,id_20,0.091889,0.198847,0.0,0.0,0.184994
3,id_24,0.094940,0.131173,0.0,0.0,0.202610


In [5]:
from datetime import datetime
import pandas as pd

def date_to_unix(date_str):
    if date_str is None:
        return None
    return int(datetime.strptime(date_str, "%d/%m/%Y").timestamp())

# Define your parameters with human-readable date strings or None
params = {
    # Text similarity inputs
    "content_query": "cinema fashion",
    "filename_query": "technical",

    # Weights for semantic spaces (matching what's in query.weights)
    "content_weight": 1.0,
    "filename_weight": 0.8,
    "size_weight": 0.5,
    "creation_date_weight": 0.6,
    "modified_date_weight": 1.0,

    # Weights for similarity matching
    "similar_content_weight": 1.0,
    "similar_filename_weight": 1.0,

    # Size filters (in kilobytes)
    "min_size_kb": 1000,
    "max_size_kb": 10000,

    # Date filters (input as "dd/mm/yyyy" strings or None)
    "min_creation_date": date_to_unix("01/01/2020"),
    "max_creation_date": date_to_unix("31/12/2024"),
    "min_modified_date": date_to_unix(None),  # No lower bound
    "max_modified_date": date_to_unix("30/06/2025"),

    # Categorical filters (must match query.py)
    "filetype_include_any": ["pdf"],
    "tags_include_any": ["cinema", "fashion"],

    # Result limit
    "limit": 5,
}

# Run the query
result = app.query(query, **params)

# Inspect parameters used
print(result.metadata.search_params)

# Convert to pandas DataFrame
df = sl.PandasConverter.to_pandas(result)

# Convert Unix timestamps in the DataFrame to dd/mm/yyyy strings for display
df["Creation_Date"] = pd.to_datetime(df["Creation_Date"], unit="s").dt.strftime("%d/%m/%Y")
df["Last_Modified_Date"] = pd.to_datetime(df["Last_Modified_Date"], unit="s").dt.strftime("%d/%m/%Y")

df


Batches: 100%|██████████| 1/1 [00:00<00:00, 91.11it/s]

12:09:02 superlinked.framework.query.query_dag_evaluator INFO   evaluated query


12:09:03 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/filesearch/points/query "HTTP/1.1 200 OK"
12:09:03 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
{'content_query': 'cinema fashion', 'similar_content_weight': 1.0, 'filename_query': 'technical', 'similar_filename_weight': 1.0, 'limit': 5, 'select_param__': ['Content', 'File_Type', 'Tags', 'Filename', 'Path', 'Size_kB', 'Creation_Date', 'Last_Modified_Date'], 'min_size_kb': 1000.0, 'max_size_kb': 10000.0, 'min_creation_date': 1577833200, 'max_creation_date': 1735599600, 'min_modified_date': None, 'max_modified_date': 1751234400, 'filetype_include_all': None, 'filetype_include_any': ['pdf'], 'filetype_exclude': None, 'tags_include_all': None, 'tags_include_any': ['cinema', 'fashion'], 'tags_exclude': None, 'natural_query': None, 'system_prompt_param__': "Extract search parameters from the user's natural language query about

,Content,File_Type,Tags,Filename,Path,Size_kB,Creation_Date,Last_Modified_Date,id,similarity_score
0,Model dress dress trend designer designer. Ten...,[pdf],"[fashion, sport]",dress.pdf,/cause/total/third/dress.pdf,6343.0,21/07/2023,05/10/2024,id_56,0.591507
1,Trend model dress trend model dress designer r...,[pdf],[fashion],trend.pdf,/seat/sometimes/painting/trend.pdf,1415.0,22/03/2020,30/06/2022,id_89,0.564794
2,Designer runway designer model model runway ru...,[pdf],"[fashion, cars, cinema]",runway.pdf,/realize/dog/increase/runway.pdf,9775.0,14/10/2021,27/03/2023,id_91,0.546632


In [ ]:
print(df.columns)
print(df.head())

In [6]:

# These are the names of the semantic/numeric spaces in the same order as in your index.spaces list
space_names = [
    "content_similarity",
    "filename_similarity",
    "size_score",
    "creation_date_score",
    "modified_date_score"
]

rows = []

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)
    
df = pd.DataFrame(rows)
df


,id,content_similarity,filename_similarity,size_score,creation_date_score,modified_date_score
0,id_56,0.101450,0.019671,0.108949,0.130219,0.231218
1,id_89,0.087603,0.041555,0.123237,0.106346,0.206054
2,id_91,0.081065,0.044035,0.089245,0.118164,0.214122
